In [ ]:
import rdflib
from rdflib.namespace import RDF, RDFS, SH, XSD
from nanopub import (
    definitions,
    Nanopub,
    NanopubClient,
    load_profile,
    NanopubConf,
)
from nanopub.fdo import (
    FdoNanopub,
    FdoRecord,
    update_record,
    retrieve_record_from_id,
    retrieve_content_from_id,
    validate_fdo_record,
    resolve_id
)
import datetime
from nanopub.namespaces import HDL, FDOF, PROV, NPX, NP
from pyshacl import validate
from rdflib import Graph, URIRef, Literal
import requests
import json


In [ ]:
## Setup

conf = NanopubConf(
    add_prov_generated_time=False,
    add_pubinfo_generated_time=True,
    attribute_assertion_to_profile=True,
    attribute_publication_to_profile=True,
    profile=load_profile(),
    use_test_server=True
)

## Define identifiers
iri = "https://w3id.org/np/RAsSeIyT03LnZt3QvtwUqIHSCJHWW1YeLkyu66Lg4FeBk/nanodash-readme"
handle_iri = "https://hdl.handle.net/21.T11966/82045bd97a0acce88378"
handle_pid = "21.T11966/82045bd97a0acce88378"


## Op.Resolve

Retrieving key-value pairs from the FDO record for given FDO PID.

In [ ]:
fdo_record_from_iri = resolve_id(iri)
fdo_record_from_handle_iri = resolve_id(handle_iri)
fdo_record_from_handle_pid = resolve_id(handle_pid)

print(fdo_record_from_iri)
print(fdo_record_from_handle_iri)
print(fdo_record_from_handle_pid)

## Op.RetrieveContent
Download the bitstream/file of an FDO.

In [ ]:
## Utility function to print the results of this operation
def print_file_info(file_content: bytes, content_type: str = "unknown", filename: str = None):
    print(f"Filename: {filename or 'N/A'}")
    print(f"Content-Type: {content_type}")
    print(f"Size: {len(file_content)} bytes")
    print("Content preview (first 500 bytes):")
    try:
        print(file_content[:500].decode("utf-8"))
    except UnicodeDecodeError:
        print(file_content[:500])  # print raw bytes if not utf-8

fdo_content_from_iri = retrieve_content_from_id(iri)
print_file_info(fdo_content_from_iri)

# TODO: replace with an FDO example with valid dataref
#fdo_content_from_handle_iri = retrieve_content_from_id(handle_iri)
#print_file_info(fdo_content_from_handle_iri)

# TODO: replace with an FDO example with valid dataref
#fdo_content_from_handle_pid = retrieve_content_from_id(handle_pid)
#print_file_info(fdo_content_from_handle_pid)

## Op.Validate
Validate given FDO record according to its profile.

In [ ]:
# Step 1: Resolve PID
fdo_record_from_iri = resolve_id(iri)

# Step 2: Validate
validation_results = validate_fdo_record(fdo_record_from_iri)
print(validation_results)

# TODO: examples with valid handle-based FDOs
# Step 1: Resolve PID
#fdo_record_from_handle_iri = resolve_id(handle_iri)

# Step 2: Validate
#validate_fdo_record(fdo_record_from_handle_iri)

# Step 1: Resolve PID
#fdo_record_from_handle_pid = resolve_id(handle_pid)

# Step 2: Validate
#validate_fdo_record(fdo_record_from_handle_pid)

## Op.Create
Defining a new FDO by creating and publishing a new FDO record (for an existing file, or without file reference).

In [ ]:
record = FdoRecord(
    profile_uri="https://hdl.handle.net/12345",
    label="Test FDO",
    dataref="https://example.org/data/456"
)

fdo_from_iri = FdoNanopub.create_with_fdo_iri(fdo_record=record, fdo_iri="https://example.org/example-fdo")
print(fdo_from_iri)

## Op.Handle2Nanopub

This operation is transforming an existing Handle-based FDO record into a nanopublication-based one. 

In [ ]:
np = FdoNanopub.handle_to_nanopub(handle_pid)

print(np)

## Op.Update record

Updates the record of an FDO, keeping the same FDO PID.

In [ ]:
# The following steps are not needed if starting from an existing nanopub, this is so that we can see a successful update (same pubkey)
# Create a new FDO record manually
initial_record = FdoRecord(
    profile_uri="https://w3id.org/fdo/profile/fdp", 
    label="Example FDO",
    dataref="https://example.org/dummy-data"
)

# Define a new FDO IRI 
fdo_iri = "https://example.org/test-fdo-123"

# Publish initial nanopub
npub = FdoNanopub.create_with_fdo_iri(
    fdo_record=initial_record,
    fdo_iri=fdo_iri,
    data_ref=initial_record.get_data_ref(),
    conf=conf
)

published_uri, server_used = npub.publish()

fetchConf = NanopubConf(
    use_test_server=True
)
fetchNp = Nanopub(published_uri, conf=fetchConf)
print(fetchNp)

# Step 1: Resolve existing record: source_uri will only be available if we find it in the nanopub network, 
# and it is used to update (supersede and re-publish) an existing nanopub, otherwise we will create a brand new nanopub
record = resolve_id(published_uri, conf=conf)

# Step 2: Make modifications
record.set_property(RDFS.label, Literal("Updated label"))

# Step 3: Update and publish
new_published_uri, server_used  = update_record(
    fdo_iri=published_uri,
    record=record,
    publish=True,
    conf=conf
)

print(f"FDO updated and published at: {new_published_uri} (server: {server_used})")

updatedNp = Nanopub(new_published_uri, conf=fetchConf)

print(updatedNp)


## Op.Create aggregation FDO

In [ ]:
fdo_iri = "https://w3id.org/fdo/example/fdo"
profile_uri= "https://w3id.org/fdo/profile/fdp"

label = "ComplexNanopubExample001"

aggregates = [
    "https://w3id.org/np/RAbb0pvoFGiNwcY8nL-qSR93O4AAcfsQRS_TNvLqt0VHg/FdoExample",
    "https://w3id.org/np/RAwCj8sM9FkB8Wyz3-i0Fh9Dcq1NniH1sErJBVEkoRQ-o/FdoExample",
    "https://w3id.org/np/RADTajQ3RJ8RNklhV8_W7B0pcJswCmm25zJPp7M-K0BRg/FdoExample",
]

npub = FdoNanopub.create_aggregation_fdo(
    fdo_iri=fdo_iri,
    profile_uri=profile_uri,
    label=label,
    aggregates=aggregates,
    conf=conf
)


npub.sign()

print(npub)
npub.publish()
